<a href="https://colab.research.google.com/github/theonlyworld/DeepLearningWithPython/blob/main/DL1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 第一眼观察神经网络

## 神经网络中的数据表现

### 使用NumPy进行张量计算
通过`train_images[i]`语句我们可以在第一个轴中选出一个特定的数字。这种在张量中挑选特定元素的行为也称作张量切片（tensor slicing）。
通过NumPy库我们可以进行切片操作，如挑选第10到100个（不包含第100个）元素，其形状是（90,28,28）


In [ ]:
my_slice = train_images[10:100]
my_slice.shape

(90, 28, 28)

下面的语句是等效的，它尤其在每一个轴中都明确了一个开始索引和结束索引。注意：号与选择该轴所有元素是相同的意思

In [ ]:
my_slice = train_images[10:100, :, :]   #与前例等同
my_slice.shape
my_slice = train_images[10:100, 0:28, 0:28] #同样与前例等同
my_slice.shape

(90, 28, 28)

你也可以在任一个轴中在两个索引中进行切片。例如，想在所有的图像中选择右下角14x14大小的区域，你可以这么做

In [ ]:
my_slice = train_images[:, 14:, 14:]

使用负索引也是支持的。原理与Python中的lists结构相似。如，要想裁切出图像中间部分14x14像素大小的区域，你可以这样做

In [ ]:
my_slice = train_images[:, 7:-7, 7:-7]

### 数据批次的概念
在深度学习中所有数据张量中的第一个轴就是样本轴（samples axis），通常也称作样本维度（samples dimension）。以MNIST为例，样本就是数据的图像。
此外，深度学习不会一次性对整个数据集进行处理，而是将数据分解为小的批次。下面这个例子就是我们MNIST数据的一个批次，大小为128。

In [ ]:
batch = train_images[:128]
batch = train_images[128:256]   #下一个批次
n = 3   #第n个批次
batch = train_images[128*n:128*(n+1)]

考虑到批次张量这一概念，第一轴也称作批次轴（batch axis）或批次维度（batch dimension）。

### 真实世界的例子
所有数据都可以分为以下几类：
- 向量 rank-2张量（样本，特征），每个样本都是一个数值属性的向量（特征）
- 时间序列或者序列数据 rank-3张量（样本，时间步长，特征），每一个样本都是一个特征向量的序列（时间步长的长度）
- 图像 rank-4张量（样本，高度，宽度，通道），每个样本都是2d的像素格，每个像素都代表一个通道数的向量
- 视频 rank-5张量（样本，帧，高度，宽度，通道），每个样本都是图像的一个序列（帧的长度）

### 向量数据
该数据集中，每个数据点可以编码成一个向量，那么一个数据的批次就会被编码为一个rank-2大小的张量（即一批向量），第一轴即样本轴，第二轴即特征轴。

### 时间序列数据或序列数据
当你的数据需要考虑时间因素时（或者顺序），那么通过加入时间轴将数据保存为rank-3形式就是必要的。每个样本可以编码为一个向量的序列（rank-2张量），那么一个批次就是一个rank-3张量。习惯上，时间轴通常是第二轴。

### 图像
图像通常有三个维度：高度，宽度，颜色。灰度图（如MNIST）只有一个颜色通道因此可以仅以rank-2的大小存储，但是习惯上图像张量总是rank-3的，以其中一个颜色通道储存灰度图像。一个128大小批次的大小为256x256的灰度图像所存储的张量大小是（128,256,256,1）而一个128批次的彩色图像的张量大小是（128,256,256,3）。

图像的张量大小有两种表达习惯：通道最后（TensorFlow）和通道最前。通道最后就是把颜色深度轴放在最后（样本，高度，宽度，颜色深度）。而通道最前就是把颜色深度轴放置到批次轴的后面（样本，颜色深度，高度，宽度）。Keras API支持两种表达习惯。

### 视频
视频是少数需要以rank-5张量存储的数据格式。视频即一系列的帧，每帧是一个彩色图。每帧可表示成一个rank-3张量（高度，宽度，颜色深度），那么一系列的帧就是一个rank-4张量（帧，高度，宽度，颜色深度），一个批次的不同视频就可以表示成一个rank-5张量（样本，帧，高度，宽度，颜色深度）。






## 神经网络的工作机制：张量操作
在最初的事例中，我们通过在每个Dense层上进行堆栈建立模型。Keras中是这样的：
`keras.layers.Dense(512, activation="relu")`
这一层可以解释成一个函数，即输入一个矩阵然后输出另一个矩阵，即输入矩阵的一种新表示。这个函数如下（W表示一个矩阵，b表示一个向量）：
`output = relu(dot(input, W) + b)`
现在让我们来解析一下这个语句，这里有三个张量操作：
- 输入张量与张量W的点乘
- 矩阵与向量b的求和
- relu操作，即`max(x,0)`

### 元素操作
relu操作和加法就是元素操作，它是在张量的单个原色的层面上进行操作的。那也就意味着这些操作是高度可并行的。如果希望使用Python编写一个元素操作的传统版本，我们需要使用到for循环，以下是relu操作的一个元素操作的朴素版本：

In [ ]:
def naive_relu(x):
    assert len(x.shape) == 2 #x是rank-2的NumPy张量
    x = x.copy() #避免覆写输入张量
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j] = max(x[i,j], 0)
    return x

加法同理：


In [ ]:
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

同理，你可以完成元素的乘法，减法等。当然这些操作在NumPy都已经了高度的集成，也是对BLAS应用的高度体现。
所以，在NumPy中，你完全可以进行以下的元素操作，而且速度惊人：

In [ ]:
import numpy as np
z = x + y #元素加法
z = np.maximum(z, 0.) #元素relu操作


时间上有什么不同呢：

In [ ]:
import time

x = np.random.random((20, 100))
y = np.random.random((20, 100))

t0 = time.time()
for _ in range(1000):
    z = x + y 
    z = np.maximum(z, 0.)
print("Took: {0:.2f} s".format(time.time() - t0))

该语句只需要0.02s,但是传统版本却需要2.45s：

In [ ]:
t0 = time.time()
for _ in range(1000):
    z = naive_add(x, y)
    z = naive_relu(z)
print("TooK: {0:.2f} s".format(time.time() - t0))

### 广播操作
之前我们编写的naive_add函数只支持特定大小的rank-2张量的加法。但是在Dense中，我们是对rank-2张量和一个向量加在一起了。这是怎么做到的呢？

通常我们会将较小的那个张量的大小扩大（广播操作）到匹配较大张量的大小。这包括两步：

1. 轴（成为扩大轴）加到较小的张量上以匹配较大张量
2. 较小的张量在新轴上不断重复以匹配完整大小的较大的张量

看个实例，设X大小为（32, 10）而y大小为（10，）：

In [ ]:
import numpy as np
x = np.random.random((32,10))
y = np.random.random((10,))

首先，我们在y中添加一个空的第一轴，则它的大小就变成了（1,10）

In [ ]:
y = np.expand_dims(y, axis=0)

然后，我们在新的轴上把y重复32次，那么我们就得到了一个大小为（32, 10）的y，而y[i, :] == y，i在range(0, 32)中：


In [ ]:
Y = np.concatenate( [y] * 32, axis=0) #将y沿着轴0重复32次

这时，我们就可以把X和Y加在一起了。

实际应用中，我们不会创造一个rank-2的张量，因为没有效率。这些重复操作在算法层面就完成了。以上的内容只是为了便于理解。这里是个传统的应用：

In [ ]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j] += y[j]
    return x

通过广播操作，就可以将大小为（a,b,...n,n+1,...m）和另一个大小为（n,n+1,...m）的张量取和。广播会在轴a到轴n-1中自动完成。
下例就是在元素层面经广播对两个不同大小的张量进行maximum操作：

In [ ]:
import numpy as np
x = np.random.random((64, 3, 32, 10))
y = np.random.random((32, 10))
z = np.maximum(x, y) #z的大小为（64,3,32,10）

### 张量乘积

张量乘积（tensor product），或称点乘（dot product），为了不与元素层面的乘积混淆，这里采用*符号表示。

在NumPy中，使用np.dot完成点乘操作：



In [ ]:
x = np.random.random((32,))
y = np.random.random((32,))
z = np.dot(x, y)

数学上，我们会表示成：
$$ z = x ⋅ y $$

那么，点乘到底做了什么？我们以两个向量间的点乘为例：

In [ ]:
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    z = 0
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z

可以看到，两个向量之间的点乘的结果是一个标量，且只有向量元素数量相同时可以点乘。

你也可以对一个矩阵x和一个向量y进行点乘，结果是一个向量，每个系数是y与x的行进行点乘得来的。如下：

In [ ]:
def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i] += x[i, j] * y[j]
    return z

需要注意，一旦两个张量中的一个ndim大于1，那么点乘就不是对称的了，也就是说dot(x, y)与dot(y,x)是不一样的。

点乘产生的张量的轴的数量是怎么确定的呢？以两个矩阵为例，对两个矩阵x和y进行点乘，只有当`x.shape[1]==y.shape[0]`时，可以进行点乘操作。结果就是一个大小为`(x.shape[0],y.shape[1])`的矩阵。矩阵中的系数来自于x中的行向量与y中的列向量的乘积。来看一下传统的版本：

In [ ]:
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]
    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            row_x = x[i, :]
            column_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, column_y)
    return z

在更高维的张量中，遵循以下和2D情形相似的规则：

$$
(a, b, c, d) ⋅ (d, ) → (a, b, c) \\
(a, b, c, d) ⋅ (d, e) → (a, b, c, e)
$$

以此类推

### 张量重塑

张量重塑在我们之前建立的第一个神经网络的Dense层中没有使用到，我们是在将数字数据输入到模型之前的预处理中使用的：

```
train_images = train_images.reshape((60000, 28 * 28)
```

重塑表示重新规定张量的行与列满足一个特定的大小。


In [ ]:
x = np.array([[0., 1.],
              [2., 3.],
              [4., 5.]])

In [ ]:
x.shape

In [ ]:
x = x.reshape((6, 1))
x

In [ ]:
x = x.reshape((2,3))
x

转置（transposition）是一种常用的重塑方法，它表示交换矩阵的行和列，即`x[i, :]`成了`x[:, i]`

In [ ]:
x = np.zeros((300,20))
x = np.transpose(x)
x.shape

### 几何解释与张量操作


每个张量可以看做是空间中的一个点，同时也表示从原点指向该点的一个箭头。张量操作的几何解释可以总结为以下几种：
- 平移
- 旋转
- 缩放
- 线性转换
- 仿射变换
- 经relu激活的Dense变换


## 神经网络的引擎：基于梯度的优化


在我们第一个模型例子中，每一层都会对输入数据进行如下的转换：

```
output = relu(dot(input, W) + b)
```

此表达式中，W和b是该层的属性张量，称为该层的权重（weights）和训练参数（trainable parameters），也称作核（kernel）与偏倚（bias）属性。他们包含了在训练数据中学习到的相关信息。

一开始，权重矩阵会填满一些随机较小的数值（随机初始化）。随后根据返回的信息逐渐修正这些权重。这个修正的过程就是学习。

以上也被称为学习循环，包含以下不断重复的主要步骤，知道损失足够低为止：

1. 获取一批训练样本x，并联系上对应的目标y_true
2. 在x上运行模型获得预测值y_pred
3. 计算损失，即y_pred与y_true之间的差异
4. 在该批次中更新权重使得损失降低

最难的部分就是第4步。如何更新权重的取值？方法就是梯度下降（gradient descent）。

### 导数（略）

### 张量操作的导数：梯度


张量操作（张量函数）的导数就是梯度。它代表了由该函数描述的高维空间表面的曲率，即输入变化时输出会如何变化。





### 随机梯度下降

设一个可微的函数，找到其最小值。理论上，其最小值所在的点的导数就是0，那么只需要找到导数为0的那些点并从中比较出最小值就可以了。

在神经网络中，我们需要解决对于$W$求解$grad(f(W),W)=0$这个问题。这是个有$N$个变量的多项式方程，变量数即模型的系数的个数。

你可以根据前述的方法来解决该问题。只要你沿梯度相反方向更新权重，损失会在每次更新后都下降一点：

1. 获取一个批次的训练样本x，以及对应的实际目标值y_true
2. 在x中运行模型，获得预测值y_pred（称为前向过程）
3. 计算批次上的损失，即y_pred与y_true间的差异大小
4. 计算模型参数条件下的损失函数的梯度（称为后向过程）
5. 将参数沿梯度相反方向调整一小部分，如`W-=learning_rate * gradient`，那么该批次的损失将会下降，而学习率（learning rate）就是调整梯度下降过程速度的一个标量。

上面描述的就是小批次随机梯度下降（mini-batch SGD），即里面每个批次的样本都是随机抽取的。如果批次大小设为1，我们一次只对一个样本进行迭代那么这种就是true SGD。又或者我们一次性对所有的全体样本进行梯度下降操作，这叫做批次梯度下降，结果很准确但是又很昂贵。所以，mini-batch SGD是两种极端互相妥协的结果。

另外，学习率的设置同样重要。数值太小，梯度下降的速度会很慢，太大就可能完全偏离最小值的区域。

当然，SGD的缺点就是无法解决局部最小的问题。为了解决这个问题，我们可以在梯度下降的过程中引入动量（momentum）这个概念。在梯度下降的过程中，同时点也在加速，即使到达了局部最小的位置，这个点也可以冲出去以到达全局最小的位置。这意味着，更新参数w不仅需要基于当前的梯度值还需要考虑之前的参数更新，如下例：

```
past_velocity = 0.
momentum = 0.1
while loss > 0.01:
    w, loss, gradient = get_current_parameters()
    velocity = past_velocity * momentum - learning_rate * gradient
    w = w + momentum * velocity - learning_rate * gradient
    past_velocity = velocity
    update_parameter(w)
```

当然，以我的理解，引入动量的概念可能也无法彻底解决局部最优的问题。

### 链接导数：反向传播算法

在现实中，要想计算一个复杂表达式的梯度可能是困难的。在这里我们就需要介绍反向传播算法（backpropagation algorithm）了。

**链原则（the chain rule）**

反向传播是通过使用一些简单操作（加、减、张量乘积等）的微分来计算由这些元操作组成的复杂组合的梯度。神经网络就是包含了很多链接在一起的张量操作，每一个都有一个简单的可知的微分。以之前构建的双层网络为例，模型是由变量W1,b1,W2,b2构成的。它包含点乘、relu、softmax以及加法等元操作。另外loss函数也是可以微分的：
```
loss_value = loss(y_true, softmax(dot(relu(dot(inputs, W1) + b1), W2) + b2))
```
微积分告诉我们一些函数的链接可根据链规则来求导。

设两个函数$f$和$g$，以及他们的组合函数$fg$，即$fg(x) == f(g(x))$:

```
def fg(x):
    x1 = g(x)
    y = f(x1)
    return y
```

根据链规则，$grad(y, x) == grad(y, x1) * grad(x1, x)$。所以只要知道$f$和$g$的微分，就可以知道$fg$的微分。这就是反向传播算法所使用的规则。接下来，我们来看看算法具体是如何工作的。

**计算图的自动微分**

通过计算图来理解反向传播会很形象。计算图构建出了一种数据结构，以及神经网络中的总体步骤。它是一种有向无环图，其表示的操作在我们这里的例子中就是张量操作。

我们先来看下面这张图:

![2.22.png](https://s2.loli.net/2023/04/09/c5pRoGyFHNCrflt.png)

这是一个我们之前建立的模型的简易版本。可以看到，图中展示的只是一个层结构，所有的变量都是标量。其中变量w和b以及输入标量x，会经过一系列操作计算出输出y。最后，计算绝对值误差损失函数：$loss_val=abs(y_true-y)$。因为我们要更新w和b以使得loss_val最小，那么我们可能就得关注$grad(loss_val, b)$和$grad(loss_val, w)$了。

前向过程，就是沿着图中的箭头进行运算的过程。

反向过程，我们需要考虑图中反向的作用大小，比如A指向B，那么B会随A变化而变化多少，也就是grad(B, A)。那么，本事例中，我们可以获得4中梯度分别是grad(loss_val, x2), grad(x2, b), grad(x2, x1), grad(x1, w)

根据链规则，在这个反向图中就可以获取一个节点对于另一节点的导数，即通过将两个节点连线的导数进行连乘。如$grad(loss_val, w)=grad(loss_val, x2)*grad(x2, x1)*grad(x1, w)$

以上就是反向传播实际运行的原理了。

**TensorFlow中的梯度tape**

TensorFlow使用自动微分法的API是GradientTape。它会记录模型中运行的张量操作，以计算图的形式保存（也称为tape）。这个图就会用来计算输出对于任一变量或变量集的梯度。变量通常是一个`tf.Variable`类。它是一个特殊的张量，意味着它的状态是可变的，如神经网络的权重系数就是`tf.Variable`实例。

In [ ]:
import tensorflow as tf
x = tf.Variable(0.)                     #实例化一个变量，初始值为0
with tf.GradientTape() as tape:         #打开一个GradientTape范围
    y = 2 * x + 2                       #在范围内对我们的变量进行一些张量操作
grad_of_y_wrt_x = tape.gradient(y, x)   #使用tape来回溯计算输出y对于变量x的梯度

`GradientTape`兼容张量操作

In [ ]:
x = tf.Variable(tf.random.uniform((2,2)))   #实例化一个大小（2,2）的变量，初始值均为0
with tf.GradientTape() as tape:
    y = 2 * x + 3
grad_of_y_wrt_x = tape.gradient(y, x)       #grad_of_y_wrt_x是大小为（2,2）的张量，它描述的是在x在[[0,0],[0,0]]周围y=2*x+3的曲率

在下面的变量中也是可行的

In [ ]:
W = tf.Variable(tf.random.uniform((2,2)))
b = tf.Variable(tf.zeros((2,)))
x = tf.random.uniform((2,2))
with tf.GradientTape() as tape:
    y = tf.matmul(x, W) + b                         #matmul即TensorFlow中的点乘操作
grad_of_y_wrt_W_and_b = tape.gradient(y, [W, b])    #grad_of_y_wrt_W_and_b是一个包含两个张量的列表，大小分别和W和b的大小相同

## 回顾第一个例子

现在，对于神经网络工作原理的一个总的认识差不多已经形成了。如图：

![2.26.png](https://s2.loli.net/2023/04/11/NDMKE8WJOfG7yiu.png)

现在我们仔细回顾一下本章开头时的第一个例子。

首先是输入数据：

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

输入数据以NumPy的张量储存，格式是float32，大小是训练数据（60000， 784），测试数据（10000,784）。

我们的模型：

In [ ]:
model = keras.Sequential([
    layers.Dense(512, activation="relu")
    layers.Dense(10, activation="softmax")
])

这个模型由两个Dense层组成的链构成。每一层都对输入的数据进行一定的张量操作，这些操作就包含了权重张量，即每一层的参数也是神经网络的“知识”的来源。

接下来就是编译的步骤：

In [ ]:
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metricx=["accuracy"])

sparse_categorical_crossentropy很容易理解就是一个损失函数了。梯度下降的一个确切规则就是名为rmsprop的优化器。

最后，就是学习循环了：

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)

当运行这一步时，模型会开始在128个训练样本的批次中进行迭代，一共进行5次（每在一个完整训练数据中迭代完一次称为一个epoch）。对于每一个批次，模型会计算损失函数对于权重的梯度大小（反向传播算法），并更新权重使得在此批次中损失函数取值更小。

在5次epoch之后，模型已经完成了2345次梯度更新了（每一epoch完成469次），且模型的损失函数大小应该已经足够小了，可以高精度地识别手写数字了。

### TensorFlow再现第一个实例

这里作为一个认识TensorFlow库的前引内容，不要求理解里面的每一步

**简易Dense类**

之前的内容已经介绍了Dense层了。这里，我们来应用一个简易的Python类，NaiveDense来创建两个TensorFlow变量W和b，并声明一个`__call__()`

In [21]:
import tensorflow as tf

class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        w_shape = (input_size, output_size)         #创建一个矩阵W，大小为（input_size， output_size），初始值为随机值
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        b_shape = (output_size,)                    #创建一个向量b，大小（output_size,）初始值为0
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):                     #前向过程
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):                               #获取层权重的简便方法
        return [self.W, self.b]

**简易Sequential类**

现在我们来创建一个`NaiveSequential`类来了链接这些层。它封装了一系列层并声明了一个`__call__()`方法来依次运行每一层。同样我们需要声明一个权重的property来轻松的追踪层中的参数。

In [32]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

通过NaiveDense和NaiveSequential类，我们就可以创建一个Keras模型了：

In [33]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

**批次生成器**

接下来，我们需要一种可以以小批次迭代遍历MNIST的方法。很简单：

In [34]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

### 运行一个训练步

这个过程中最困难的部分就是“训练步”：在一个批次数据运行模型后更新权重。我们需要

1. 计算该批次模型对于图像的预测值
2. 基于真实标签，计算这些预测值的损失函数
3. 计算损失函数对于模型权重的梯度
4. 小幅度更新权重

为了更新梯度，我们要使用TensorFlow中的GradientTape：

In [35]:
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:                         #运行前向过程，在GradientTape范围内计算预测值
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)  #计算损失函数的梯度。输出是一个列表，每一个entry对应模型权重列表的一个权重
    update_weights(gradients, model.weights)                #使用梯度更新权重，这个函数我待会来定义
    return average_loss

更新权重就是超一个特定方向小幅度更新权重，更新的幅度就是“学习率”。那么实施权重更新的最简单的方法就是对每个权重减去梯度与学习率的乘积：

In [36]:
learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)     #assign_sub对于TensorFlow变量等价于-=

实践中，并不需要如此更新权重。只需要keras中的`Optimizer`实例即可：

In [37]:
from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate = 1e-3)

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

现在每个批次的训练步骤都准备好了，可以完成下面的整个数据的训练了

### 完整的训练循环

一个epoch的训练包含了训练数据中每个批次的重复的训练步。总的训练循环就是一个epoch的简单重复：

In [38]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")

现在我们来尝试运行一下：

In [ ]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

### 评价我们的模型

通过argmax来

In [42]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.81
